<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=22b64d96481f320e670fe58f8d7aba07dc43918ab4e5306b21daae038e219ec5
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-19 14:30:06
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.40 C
-------------------
Today PnL: 66.56 K (0.48%)
Current PnL: -27.14 L (-17.53%)
CY Booked + Current PnL: -12.85 L (-8.3%)
-------------------
Total profit:  1.36 L
Total loss:  -28.49 L
-------------------
Total Booked + Current PnL: 14.13 L (11.06%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.21%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 91.77 L (65.59%)
Deployed:  1.28 C
Current:  1.40 C
CAGR/XIRR %: 5.01%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-0.29,-20.38,25.71,0.09,20629.0,-20539.0,80238.0,86.34,26.0,M-SC,2.31,253.0,-1.00,0.58,5.41,OX40N,NTT,DURABLES
82,VOLTAS,1530.00,-2.33,7.10,11.76,19.69,24149.0,13608.0,205350.0,-3.29,50.0,X-MC,2.50,78.0,0.56,1.47,14.38,XY25,NTT,AC
50,MASFIN,398.61,1.50,-2.88,25.67,22.05,24428.0,-2820.0,95160.0,-16.44,52.0,H-SC,6.41,164.0,-0.12,0.68,38.32,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,0.29,-16.03,19.11,0.02,28011.0,-27979.0,146580.0,123.77,42.0,M-SC,13.11,234.0,-1.00,1.05,27.50,OX40N,NTT,PAINTS
43,ITC,452.00,0.15,-1.86,12.89,10.78,30399.0,-4482.0,235836.0,-41.56,43.0,X-LC,2.06,5.0,-0.15,1.69,3.27,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAELXSI,9161.00,8.22,-21.81,68.85,32.03,70973.0,-28751.0,103084.0,-15.37,63.0,H-SC,9.51,148.0,-0.41,0.74,15.46,OX40N,NTT,IT
6,ASIANTILES,137.00,4.25,-5.05,91.34,81.67,81357.0,-4740.0,89070.0,8016.67,70.0,L-SC,12.99,271.0,-0.06,0.64,72.49,XR,NTT,CERAMICS
64,SHALBY,327.00,3.64,-23.89,64.36,25.10,88734.0,-43271.0,137872.0,914.35,42.0,M-SC,10.98,250.0,-0.49,0.99,10.49,XY24,NTT,HEALTHCARE
10,BANDHANBNK,400.00,2.96,-28.02,167.33,92.42,335503.0,-78055.0,200504.0,40.80,51.0,H-SC,9.25,170.0,-0.23,1.44,15.88,XY24,NTT,BANKS
67,SONACOMS,806.63,2.42,-13.81,61.91,39.54,53976.0,-13974.0,87185.0,-32.63,55.0,M-SC,4.15,220.0,-0.26,0.63,22.94,AR,ATH,AUTO


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
71,TANLA,1943.92,-3.20,-38.27,255.90,119.69,427704.0,-103624.0,167137.0,-37.58,40.0,H-SC,7.50,156.0,-0.24,1.20,32.04,AR,ATH,IT
82,VOLTAS,1530.00,-2.33,7.10,11.76,19.69,24149.0,13608.0,205350.0,-3.29,50.0,X-MC,2.50,78.0,0.56,1.47,14.38,XY25,NTT,AC
38,INDIAMART,4810.62,-2.08,-4.73,116.99,106.72,137464.0,-5835.0,117501.0,-52.92,36.0,H-SC,3.68,139.0,-0.04,0.84,17.54,AR,ATH,MISC
61,ROUTE,2227.21,-1.22,-45.27,208.91,69.05,152130.0,-60245.0,72821.0,-55.87,57.0,H-SC,21.95,140.0,-0.40,0.52,12.31,SR,ATH,IT
76,TRIDENT,48.00,-1.22,-28.59,79.98,28.51,52687.0,-26380.0,65875.0,-15.53,32.0,M-SC,4.34,225.0,-0.50,0.47,12.72,XR,NTT,TEXTILES


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,1.23,-0.66,109.29,107.91,160299.0,-971.0,146673.0,-19.34,60.0,M-SC,12.28,216.0,-0.01,1.05,9.36,OX40N,ATH,CABLES


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,1.23,-0.66,109.29,107.91,160299.0,-971.0,146673.0,-19.34,60.0,M-SC,12.28,216.0,-0.01,1.05,9.36,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.29,-16.03,19.11,0.02,28011.0,-27979.0,146580.0,123.77,42.0,M-SC,13.11,234.0,-1.00,1.05,27.50,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.29,-20.38,25.71,0.09,20629.0,-20539.0,80238.0,86.34,26.0,M-SC,2.31,253.0,-1.00,0.58,5.41,OX40N,NTT,DURABLES
72,TATAELXSI,9161.00,8.22,-21.81,68.85,32.03,70973.0,-28751.0,103084.0,-15.37,63.0,H-SC,9.51,148.0,-0.41,0.74,15.46,OX40N,NTT,IT
66,SIS,528.00,0.65,-23.75,59.04,21.26,49983.0,-26372.0,84660.0,1997.93,52.0,H-SC,4.83,166.0,-0.53,0.61,14.38,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.28,8.53,58.73,72.27,108003.0,14449.0,183897.0,-6.99,70.0,M-LC,3.06,99.0,0.13,1.32,15.23,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.28,8.53,58.73,72.27,108003.0,14449.0,183897.0,-6.99,70.0,M-LC,3.06,99.0,0.13,1.32,15.23,XR,NTT,IT
25,FINCABLES,1641.55,1.23,-0.66,109.29,107.91,160299.0,-971.0,146673.0,-19.34,60.0,M-SC,12.28,216.0,-0.01,1.05,9.36,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-2.08,-4.73,116.99,106.72,137464.0,-5835.0,117501.0,-52.92,36.0,H-SC,3.68,139.0,-0.04,0.84,17.54,AR,ATH,MISC
6,ASIANTILES,137.00,4.25,-5.05,91.34,81.67,81357.0,-4740.0,89070.0,8016.67,70.0,L-SC,12.99,271.0,-0.06,0.64,72.49,XR,NTT,CERAMICS
37,IEX,219.00,0.88,-4.32,55.33,48.63,105859.0,-8631.0,191323.0,-36.24,48.0,H-SC,13.54,136.0,-0.08,1.37,6.74,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,PAGEIND,51605.07,0.22,-11.19,44.25,28.11,79152.0,-22535.0,178875.0,-35.61,25.0,X-MC,13.78,55.0,-0.28,1.28,0.22,X40,ATH,APPARELS
9,BAJAJHFL,181.50,0.11,-18.50,90.77,55.47,179971.0,-45014.0,198272.0,-27.82,27.0,X-MC,14.38,64.0,-0.25,1.42,1.59,X40N,ATH,FINANCE
8,AWL,485.00,0.44,-23.95,101.83,53.50,233686.0,-72265.0,229486.0,-63.25,30.0,X-MC,4.21,58.0,-0.31,1.65,1.93,XY24,NTT,FMCG
11,BATAINDIA,2096.00,0.57,-39.01,121.68,35.20,95490.0,-50194.0,78476.0,2.96,33.0,X-SC,16.61,92.0,-0.53,0.56,0.66,X40,NTT,FOOTWEAR
33,HINDUNILVR,2922.00,0.68,-11.27,28.10,13.67,64096.0,-28963.0,228100.0,-17.95,34.0,X-LC,2.60,14.0,-0.45,1.64,7.68,XY25,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,950.0,0.19,-1.24,26.04,24.48,64767.0,-3135.0,248721.0,-81.09,46.0,X-SC,0.77,86.0,-0.05,1.79,21.61,X40N,NTT,MISC
12,BERGEPAINT,680.0,0.40,-4.07,26.28,21.14,57315.0,-9247.0,218092.0,-10.87,40.0,X-MC,1.34,74.0,-0.16,1.57,22.93,XY24,NTT,PAINTS
20,DABUR,735.0,0.09,-3.61,49.09,43.70,117377.0,-8963.0,239105.0,-11.62,34.0,X-MC,2.02,72.0,-0.08,1.72,10.89,XY24,BTT,FMCG
43,ITC,452.0,0.15,-1.86,12.89,10.78,30399.0,-4482.0,235836.0,-41.56,43.0,X-LC,2.06,5.0,-0.15,1.69,3.27,X40,NTT,FMCG
78,UNITDSPR,1644.0,1.06,3.28,16.94,20.77,40963.0,7671.0,241815.0,-5.65,44.0,X-MC,2.27,62.0,0.19,1.74,10.42,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,PAGEIND,51605.07,0.22,-11.19,44.25,28.11,79152.0,-22535.0,178875.0,-35.61,25.0,X-MC,13.78,55.0,-0.28,1.28,0.22,X40,ATH,APPARELS
3,ACC,3906.00,0.64,-25.77,121.15,64.15,213975.0,-61331.0,176620.0,-56.75,36.0,X-SC,3.18,82.0,-0.29,1.27,0.64,XY24,BTT,CEMENT
19,COLPAL,3726.84,0.51,-20.24,77.41,41.51,162615.0,-53295.0,210070.0,-4.98,38.0,X-MC,7.63,61.0,-0.33,1.51,0.64,XY25,ATH,FMCG
11,BATAINDIA,2096.00,0.57,-39.01,121.68,35.20,95490.0,-50194.0,78476.0,2.96,33.0,X-SC,16.61,92.0,-0.53,0.56,0.66,X40,NTT,FOOTWEAR
59,RELAXO,1176.00,-0.33,-48.02,193.01,52.31,145634.0,-69706.0,75454.0,-44.63,41.0,X-SC,7.30,91.0,-0.48,0.54,1.00,X40N,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,1.37,-29.75,103.34,42.84,47191.0,-19340.0,45666.0,-54.27,44.0,X-SC,34.78,83.0,-0.41,0.33,3.31,XY24,NTT,MISC
59,RELAXO,1176.00,-0.33,-48.02,193.01,52.31,145634.0,-69706.0,75454.0,-44.63,41.0,X-SC,7.30,91.0,-0.48,0.54,1.00,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.57,-39.01,121.68,35.20,95490.0,-50194.0,78476.0,2.96,33.0,X-SC,16.61,92.0,-0.53,0.56,0.66,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,1.67,-2.16,26.81,24.08,33301.0,-2739.0,124211.0,-10.14,43.0,X-SC,5.53,89.0,-0.08,0.89,17.03,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,1.00,-16.60,71.84,43.32,97587.0,-27032.0,135840.0,-28.31,39.0,X-SC,5.05,90.0,-0.28,0.98,4.30,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,0.47,-9.37,33.18,20.71,113743.0,-35432.0,342805.0,-22.01,70.0,X-LC,3.87,1.0,-0.31,2.46,14.54,X40,ATH,IT
41,INFY,1972.00,0.33,11.11,20.82,34.24,72383.0,34771.0,347659.0,-13.50,67.0,X-LC,4.86,2.0,0.48,2.50,20.32,X40,NTT,IT
75,TMPV,600.00,1.71,-27.92,70.60,22.98,114963.0,-63061.0,162837.0,-25.52,40.0,X-LC,6.83,3.0,-0.55,1.17,2.42,XY24,NTT,AUTO
81,VBL,671.64,-0.95,-5.24,43.18,35.67,129232.0,-16556.0,299286.0,-16.55,48.0,X-LC,3.94,4.0,-0.13,2.15,7.89,X40N,ATH,FMCG
43,ITC,452.00,0.15,-1.86,12.89,10.78,30399.0,-4482.0,235836.0,-41.56,43.0,X-LC,2.06,5.0,-0.15,1.69,3.27,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.94,-37.78,113.59,32.89,54565.0,-29170.0,48037.0,-693.00,53.0,L-MC,6.34,259.0,-0.53,0.34,32.29,XR,NTT,BANKS
6,ASIANTILES,137.00,4.25,-5.05,91.34,81.67,81357.0,-4740.0,89070.0,8016.67,70.0,L-SC,12.99,271.0,-0.06,0.64,72.49,XR,NTT,CERAMICS
50,MASFIN,398.61,1.50,-2.88,25.67,22.05,24428.0,-2820.0,95160.0,-16.44,52.0,H-SC,6.41,164.0,-0.12,0.68,38.32,XR,ATH,FINANCE
13,BSOFT,831.70,0.12,-21.34,91.70,50.79,101450.0,-30021.0,110632.0,0.89,66.0,H-SC,6.88,151.0,-0.30,0.79,28.81,XR,ATH,IT
39,INDIGOPNTS,1408.00,0.29,-16.03,19.11,0.02,28011.0,-27979.0,146580.0,123.77,42.0,M-SC,13.11,234.0,-1.00,1.05,27.50,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,0.12,-21.34,91.70,50.79,101450.0,-30021.0,110632.0,0.89,66.0,H-SC,6.88,151.0,-0.30,0.79,28.81,XR,ATH,IT
84,WIPRO,420.00,0.28,8.53,58.73,72.27,108003.0,14449.0,183897.0,-6.99,70.0,M-LC,3.06,99.0,0.13,1.32,15.23,XR,NTT,IT
6,ASIANTILES,137.00,4.25,-5.05,91.34,81.67,81357.0,-4740.0,89070.0,8016.67,70.0,L-SC,12.99,271.0,-0.06,0.64,72.49,XR,NTT,CERAMICS
61,ROUTE,2227.21,-1.22,-45.27,208.91,69.05,152130.0,-60245.0,72821.0,-55.87,57.0,H-SC,21.95,140.0,-0.40,0.52,12.31,SR,ATH,IT
41,INFY,1972.00,0.33,11.11,20.82,34.24,72383.0,34771.0,347659.0,-13.50,67.0,X-LC,4.86,2.0,0.48,2.50,20.32,X40,NTT,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.37
1,25,44.86
2,50,76.59


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.91
MC    30.84
LC    24.23
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.72
X40      23.10
X40N     12.99
XR        9.80
XY25      9.07
AR        8.96
OX40N     7.57
SR        1.01
MH        0.76
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.37
H-SC    23.72
X-LC    20.78
M-SC    11.63
X-SC     8.08
H-MC     4.79
L-SC     1.48
M-MC     1.34
M-LC     1.32
H-LC     1.11
L-LC     1.02
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.29,-8.41,43.27
IT,13.17,-14.61,72.24
FINANCE,9.45,-20.89,73.47
MISC,7.26,-28.42,80.96
ELECTRICAL,5.99,-12.53,53.75
PAINTS,5.83,-13.78,30.68
INSURANCE,4.80,-1.40,36.10
PHARMA,4.12,-2.46,34.71
AUTO,2.86,-31.92,77.65


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3200283.0,21
AR,1324679.0,10
XR,1295338.0,13
X40,1005495.0,14
X40N,876968.0,9
OX40N,737874.0,10
XY25,378725.0,6
SR,278952.0,2
MH,78918.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3674278.0,25
M-SC,1439116.0,15
X-MC,1433715.0,16
X-LC,831427.0,11
X-SC,795014.0,8
H-MC,410815.0,3
L-SC,255512.0,3
M-LC,108003.0,1
H-LC,78593.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1287455.0      6
           AR         905747.0      5
M-SC       XY24       822800.0      6
H-SC       XR         787147.0      7
X-MC       X40        493507.0      7
           XY24       408378.0      3
           X40N       345066.0      4
H-SC       OX40N      336059.0      4
X-LC       X40        319429.0      5
M-SC       OX40N      315692.0      5
X-SC       X40N       307988.0      3
           XY24       294467.0      3
H-SC       SR         278952.0      2
X-LC       X40N       223914.0      2
H-MC       AR         215949.0      2
           XY24       194866.0      1
X-SC       X40        192559.0      2
X-LC       XY24       192317.0      2
X-MC       XY25       186764.0      2
M-SC       XR         176234.0      2
L-SC       XR         169389.0      2
M-SC       AR         124390.0      2
M-LC       XR         108003.0      1
X-LC       XY25        95767.0      2
L-SC       OX40N       86123.0      1
H-SC       MH          78918.0      1
H-LC       AR          78593.0      1
M-MC       XY25        59281.0      1
L-MC       XR          54565.0      1
L-LC       XY25        36913.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 39.0 seconds
